In [45]:
import json
from pathlib import Path
from soundevent import data, io
from collections import defaultdict
from tensorflow._api.v2.lite import Interpreter
from audioclass import SimpleIterator
from audioclass.models.birdnet import BirdNET
import pandas as pd
from tqdm import tqdm

In [ ]:
# Load in the annotation project

In [6]:
fp1_annotation_path = Path("SM03_6s_2024-07-18T00_26_01.625827.json")
with open(fp1_annotation_path) as file:
    annotation_contents = json.load(file)

print(json.dumps(annotation_contents, indent=2))

{
  "version": "1.1.0",
  "created_on": "2024-07-18T00:26:01.625827",
  "data": {
    "uuid": "1ac585ff-5e7b-440b-b45f-b12985259a3c",
    "collection_type": "annotation_project",
    "users": [
      {
        "uuid": "63e6952b-1e95-440b-9fbb-6e0aa03b4ecb",
        "username": "hhhyt",
        "email": "yuting.huang.23@ucl.ac.uk",
        "name": "Yuting Huang",
        "institution": null
      }
    ],
    "tags": [
      {
        "id": 0,
        "key": "sound",
        "value": "bird"
      },
      {
        "id": 1,
        "key": "environment",
        "value": "rain or wind"
      },
      {
        "id": 2,
        "key": "sound",
        "value": "unknown"
      },
      {
        "id": 3,
        "key": "n/a",
        "value": "nothing"
      },
      {
        "id": 4,
        "key": "sounds",
        "value": "parrot"
      },
      {
        "id": 5,
        "key": "environment",
        "value": "drops"
      },
      {
        "id": 6,
        "key": "parrot",
        

In [10]:
fp1_sample = io.load(fp1_annotation_path, type="annotation_project")
print(repr(fp1_sample))

AnnotationProject(created_on=datetime.datetime(2024, 5, 30, 15, 29, 53, 294422), name='SM03_6s', description='The clips that requires annotations for the project from the device sm03', instructions='2 clips of 6s extracted from each recording')


In [14]:
# view the clip annotations
for clip_annotation in fp1_sample.clip_annotations:
    clip = clip_annotation.clip
    recording = clip.recording
    print(
        f"* Recording {recording.path} [from "
        f"{clip.start_time:.3f}s to {clip.end_time:.3f}s]"
    )
    print(
        f"\t{len(clip_annotation.sound_events)} sound event annotations found"
    )
    for annotation in clip_annotation.sound_events:
        sound_event = annotation.sound_event
        print(f"\t+ Sound event ")
        for tag in annotation.tags:
            print(f"\t\t- {tag}")
    print("")

* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM03 (FP1)\2022\SM03_01_2022\FP1_20220119_180000.wav [from 96.000s to 102.000s]
	0 sound event annotations found

* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM03 (FP1)\2021\SM03_03_2021\FP1_20210326_060000.wav [from 258.000s to 264.000s]
	0 sound event annotations found

* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM03 (FP1)\2019\SM03_06_2019\SM03_20190610_150000.wav [from 120.000s to 126.000s]
	0 sound event annotations found

* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM03 (FP1)\2019\SM03_05_2019\SM03_20190518_090000.wav [from 108.000s to 114.000s]
	0 sound event annotations found

* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM03 (FP1)\2020\SM03_06_2020\FP1_20200606_060000.wav [from 234.000s to 240.000s]
	0 sound event annotations found

* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM03 (FP1)\2021\SM03_01_2021\FP1_20210112_1000

In [16]:
# save the annotation project
io.save(fp1_sample, "fp1_sample.json")

In [18]:
# 2) select the annotated clips

In [18]:
# create functions
def load_annotation_file(annotation_file):
    return io.load(annotation_file, type="annotation_project")


def has_issue(task):
    for badge in task.status_badges:
        if badge.state == data.AnnotationState.rejected:
            return True

    return False


def is_complete(task):
    for badge in task.status_badges:
        if badge.state == data.AnnotationState.completed:
            return True

    return False


def select_clips(annotation_project, complete=None, issue=None):
    tasks = [task for task in annotation_project.tasks]

    if complete is not None:
        tasks = [task for task in tasks if is_complete(task) == complete]

    if issue is not None:
        tasks = [task for task in tasks if has_issue(task) == issue]

    clips = {task.clip.uuid for task in tasks}
    return [
        clip_annotation
        for clip_annotation in annotation_project.clip_annotations
        if clip_annotation.clip.uuid in clips
    ]

path = "fp1_sample.json"
annotation_project = load_annotation_file(path)
complete_clips = select_clips(annotation_project, complete=True, issue=False)

In [20]:
# view the completely annotated clips
for clip_annotation in complete_clips:
    clip = clip_annotation.clip
    recording = clip.recording
    print(
        f"* Recording {recording.path} [from "
        f"{clip.start_time:.3f}s to {clip.end_time:.3f}s]"
    )

* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM03 (FP1)\2022\SM03_01_2022\FP1_20220119_180000.wav [from 96.000s to 102.000s]
* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM03 (FP1)\2021\SM03_03_2021\FP1_20210326_060000.wav [from 258.000s to 264.000s]
* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM03 (FP1)\2019\SM03_06_2019\SM03_20190610_150000.wav [from 120.000s to 126.000s]
* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM03 (FP1)\2019\SM03_05_2019\SM03_20190518_090000.wav [from 108.000s to 114.000s]
* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM03 (FP1)\2020\SM03_06_2020\FP1_20200606_060000.wav [from 234.000s to 240.000s]
* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM03 (FP1)\2021\SM03_01_2021\FP1_20210112_100000.wav [from 216.000s to 222.000s]
* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM03 (FP1)\2019\SM03_03_2019\TEST-B_20190326_100000.wav [from 186.000s t

In [22]:
len(complete_clips)

998

In [47]:
# 3) Extract features
 # Load latest annotations
annotation_project = io.load("fp1_sample.json")

In [49]:
# Load the birdnet model into the notebook!
model = BirdNET.load()

In [51]:
# as all 990 clips have been completely annotated, no need to remove any clips from the original json
selected_clip_annotations = annotation_project.clip_annotations[:998]

In [53]:
# check the audio dir and path
prefix = Path.home() / "OneDrive" / "桌面" / "seychelles_parrot_data"
print(f"Prefix exists: {prefix.exists()}")
audio_dir = Path("SM03 (FP1)")
print(f"audio dir: {audio_dir.exists()}")

Prefix exists: True
audio dir: False


In [55]:
# fix the path

for clip_annotation in selected_clip_annotations:
    clip = clip_annotation.clip
    recording = clip.recording

    # Only fix the path if not previously done
    recording.path = Path(str(recording.path).replace("妗岄潰","桌面"))

    if not recording.path.exists():
        raise ValueError(f"File not found {recording.path}")

In [57]:
# view the infos using an example clip
example_clip = selected_clip_annotations[0].clip

print("Clips contain info about when they start and end: ", clip.start_time, clip.end_time)

example_outputs = model.process_clip(example_clip)

print("The output contains two processed clips", len(example_outputs))

example_processed_clip = example_outputs[0]

print("Processed clips might be a subclip of the input: ", example_processed_clip.clip.start_time, example_processed_clip.clip.end_time) 

print("Processed clips can contain extracted features. In this case: ", len(example_processed_clip.features))

print("The first feature:  ", example_processed_clip.features[0])

Clips contain info about when they start and end:  78.0 84.0
The output contains two processed clips 2
Processed clips might be a subclip of the input:  96.0 99.0
Processed clips can contain extracted features. In this case:  1024
The first feature:   name='BirdNET_0' value=0.0511409156024456


In [59]:
# Process all clips in a loop
def extract_tags_info(tags):
    return {tag.value: True for tag in tags}


def extract_task_info(annotation_task):
    ready = False
    issues = False

    for badge in annotation_task.status_badges:
        if badge.state == data.AnnotationState.completed:
            ready = True
            continue

        if badge.state == data.AnnotationState.rejected:
            issues = True
            continue

    return {"ready": ready, "issues": issues}


def extract_clip_annotation_info(clip_annotation):
    return {
        "annotation_id": str(clip_annotation.uuid),
        **extract_tags_info(clip_annotation.tags),
    }


def extract_feature_info(processed_clip):
    return {
        "path": processed_clip.clip.recording.path,
        "clip_id": processed_clip.clip.uuid,
        "start_time": processed_clip.clip.start_time,
        "end_time": processed_clip.clip.end_time,
        **{feature.name: feature.value for feature in processed_clip.features},
    }


# Process all clips in a loop
outputs = []
for clip_annotation in tqdm(selected_clip_annotations):
    clip = clip_annotation.clip

    # Its a good idea to wrap this processing bit in a try-except block
    # so that we can safely skip the clips that failed to process.
    try:
        processed_clips = model.process_clip(clip)

        # NOTE: If the clip is long the model will split it into multiple clips
        # and return a list of processed clips.
        for processed_clip in processed_clips:
            outputs.append(
                {
                    **extract_feature_info(processed_clip),
                    **extract_clip_annotation_info(clip_annotation),
                }
            )

    except Exception as e:
        print(f"Failed to process clip: {clip}. Error: {e}")
        continue

100%|████████████████████████████████████████████████████████████████████████████████| 998/998 [03:01<00:00,  5.49it/s]


In [61]:
# creat a pandas dataframe 
df = pd.DataFrame(outputs)

In [63]:
df

,path,clip_id,start_time,end_time,BirdNET_0,BirdNET_1,BirdNET_2,BirdNET_3,BirdNET_4,BirdNET_5,...,annotation_id,bird,rain or wind,unknown,nothing,parrot,drops,whistle,sunbird,frog
0,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,3a239f3a-d947-4d10-8889-f0950b69b260,96.0,99.0,0.051141,0.546658,0.086821,0.180957,0.254434,0.152166,...,c9ac7b15-b19d-4226-b91a-2b19678a6df5,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,8146293e-129e-4cbe-878d-416e6defa38c,99.0,102.0,0.000000,0.043104,0.104107,0.177910,0.821513,0.050157,...,c9ac7b15-b19d-4226-b91a-2b19678a6df5,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,04881e49-e918-41aa-9ade-5c5b44e3ccf2,258.0,261.0,0.012194,0.029567,0.054304,0.055651,0.071335,0.589626,...,9c2f1e49-8cc5-4eac-ac93-3f27f7ec854e,NaN,True,True,NaN,NaN,NaN,NaN,NaN,NaN
3,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,4eabeb86-3baf-4e48-879b-feb94e71e723,261.0,264.0,0.199227,0.419799,0.031437,0.379074,0.160746,0.228738,...,9c2f1e49-8cc5-4eac-ac93-3f27f7ec854e,NaN,True,True,NaN,NaN,NaN,NaN,NaN,NaN
4,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,23038cf5-d320-47cf-9734-30e24e4cae0b,120.0,123.0,0.083655,0.505305,0.097888,0.283815,0.286338,0.000000,...,c5a66122-3b61-4eb4-9cc7-6f0790454b15,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1991,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,751b8a06-37f6-41f9-9400-ea74b5db556d,177.0,180.0,0.000000,0.480020,0.117134,0.381812,0.333140,0.061873,...,570f6859-6ec5-499a-93d6-51408beab34c,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,0ea298a0-d9e7-4c84-9ae4-6eff0ed9fa19,228.0,231.0,0.025179,0.676086,0.040974,0.317406,0.221856,0.083549,...,d1026f5d-8033-4f8d-9eac-9e77508a4f0f,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
1993,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,6594bc64-1154-4ee9-b162-ace1888fe6c9,231.0,234.0,0.048810,0.438392,0.127954,0.277786,0.370167,0.478421,...,d1026f5d-8033-4f8d-9eac-9e77508a4f0f,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
1994,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,41bb8dba-b84e-495f-9cc3-06ad0110a6e7,78.0,81.0,0.120349,0.171664,0.127131,0.772129,0.349963,0.186776,...,9610169b-d12e-4178-b6fe-e3d77fe4d436,NaN,True,NaN,True,NaN,NaN,NaN,NaN,NaN


In [65]:
df.to_csv("full_fp1_features.csv")

In [69]:
# compute the average features for two subclips
full_fp1_features = pd.read_csv('full_fp1_features.csv', low_memory=False)
full_fp1_features.drop(columns = ['clip_id', 'Unnamed: 0'])

,path,start_time,end_time,BirdNET_0,BirdNET_1,BirdNET_2,BirdNET_3,BirdNET_4,BirdNET_5,BirdNET_6,...,annotation_id,bird,rain or wind,unknown,nothing,parrot,drops,whistle,sunbird,frog
0,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,96.0,99.0,0.051141,0.546658,0.086821,0.180957,0.254434,0.152166,0.987146,...,c9ac7b15-b19d-4226-b91a-2b19678a6df5,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,99.0,102.0,0.000000,0.043104,0.104107,0.177910,0.821513,0.050157,0.614261,...,c9ac7b15-b19d-4226-b91a-2b19678a6df5,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,258.0,261.0,0.012194,0.029567,0.054304,0.055651,0.071335,0.589626,0.076262,...,9c2f1e49-8cc5-4eac-ac93-3f27f7ec854e,NaN,True,True,NaN,NaN,NaN,NaN,NaN,NaN
3,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,261.0,264.0,0.199227,0.419799,0.031437,0.379074,0.160746,0.228738,0.651410,...,9c2f1e49-8cc5-4eac-ac93-3f27f7ec854e,NaN,True,True,NaN,NaN,NaN,NaN,NaN,NaN
4,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,120.0,123.0,0.083655,0.505305,0.097888,0.283815,0.286338,0.000000,1.046726,...,c5a66122-3b61-4eb4-9cc7-6f0790454b15,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1991,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,177.0,180.0,0.000000,0.480020,0.117134,0.381812,0.333140,0.061873,0.806639,...,570f6859-6ec5-499a-93d6-51408beab34c,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,228.0,231.0,0.025179,0.676086,0.040974,0.317406,0.221856,0.083549,1.011195,...,d1026f5d-8033-4f8d-9eac-9e77508a4f0f,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
1993,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,231.0,234.0,0.048810,0.438392,0.127954,0.277786,0.370167,0.478421,0.840103,...,d1026f5d-8033-4f8d-9eac-9e77508a4f0f,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
1994,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,78.0,81.0,0.120349,0.171664,0.127131,0.772129,0.349963,0.186776,0.661531,...,9610169b-d12e-4178-b6fe-e3d77fe4d436,NaN,True,NaN,True,NaN,NaN,NaN,NaN,NaN


In [71]:
full_fp1_features.columns

Index(['Unnamed: 0', 'path', 'clip_id', 'start_time', 'end_time', 'BirdNET_0',
       'BirdNET_1', 'BirdNET_2', 'BirdNET_3', 'BirdNET_4',
       ...
       'annotation_id', 'bird', 'rain or wind', 'unknown', 'nothing', 'parrot',
       'drops', 'whistle', 'sunbird', 'frog'],
      dtype='object', length=1039)

In [73]:
# Define columns that are categorical (factors)
categorical_columns = ['path', 'bird', 'rain or wind', 'unknown', 'nothing', 'parrot',
       'drops', 'whistle', 'sunbird', 'frog']  # Add any other categorical columns here

# Identify numeric columns for averaging
numeric_columns = full_fp1_features.select_dtypes(include=[float, int]).columns

# Remove categorical and time columns from numeric columns list if they are included
numeric_columns = [col for col in numeric_columns if col not in categorical_columns + ['start_time', 'end_time','annotation_id']]

# Group by 'annotation_id' and compute the average of each numeric feature column
fused_features_numeric = full_fp1_features.groupby('annotation_id')[numeric_columns].mean().reset_index()

# For categorical columns, we can keep the first instance (assuming consistency)
# Exclude 'annotation_id' here to avoid duplication
fused_features_categorical = full_fp1_features.groupby('annotation_id')[categorical_columns].first().reset_index()

# For start_time and end_time,custom logic
start_times = full_fp1_features.groupby('annotation_id')['start_time'].first().reset_index()
end_times = full_fp1_features.groupby('annotation_id')['end_time'].last().reset_index()

# Merge numeric, categorical, and time DataFrames
fused_features = fused_features_numeric.merge(fused_features_categorical, on='annotation_id')
fused_features = fused_features.merge(start_times, on='annotation_id')
fused_features = fused_features.merge(end_times, on='annotation_id')
fused_features.drop(columns = ['Unnamed: 0'])

,annotation_id,BirdNET_0,BirdNET_1,BirdNET_2,BirdNET_3,BirdNET_4,BirdNET_5,BirdNET_6,BirdNET_7,BirdNET_8,...,rain or wind,unknown,nothing,parrot,drops,whistle,sunbird,frog,start_time,end_time
0,003e93f7-cf43-40b9-8373-5d43c50871a8,0.077048,0.245560,0.255525,0.037137,0.617168,0.078559,0.528461,0.100497,0.186559,...,True,None,None,True,None,None,None,None,282.0,288.0
1,00a47c33-c5fd-4a94-8f43-dd118660dde3,0.024353,0.668725,0.175876,0.258857,0.166176,0.081506,1.130004,0.199331,0.508516,...,None,None,None,True,None,None,None,None,126.0,132.0
2,00c112fa-a272-484a-b7dc-ed242eb3af78,0.089544,0.452753,0.007244,0.233920,0.265389,0.122408,1.041941,0.143856,0.454708,...,True,True,None,None,None,None,None,None,42.0,48.0
3,00d15e74-4ef8-445c-beff-d70c522640d7,0.139818,0.285597,0.051643,0.223714,0.291120,0.191062,1.089246,0.092381,0.409825,...,True,None,True,None,None,None,None,None,24.0,30.0
4,0100972b-1c98-4089-b149-826f5d6d4808,0.084454,0.433399,0.059982,0.311007,0.316492,0.190456,0.915934,0.322653,0.485731,...,None,None,True,None,None,None,None,None,36.0,42.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993,fe4bf03c-db79-46bb-9f2e-d98523cddcde,0.003800,0.000000,0.184203,0.448457,0.917729,0.514493,0.364949,0.055889,0.288647,...,True,True,None,None,None,None,None,None,96.0,102.0
994,fed25711-b2ae-432a-81e1-9a590d0a009e,0.103586,0.194146,0.178390,0.753330,0.427396,0.397441,0.545625,0.109222,0.447043,...,None,True,None,None,None,None,None,None,204.0,210.0
995,ff15e62d-656e-4125-a476-dd6af2e20b6e,0.073153,0.489372,0.079489,0.416042,0.280941,0.060840,1.065122,0.234270,0.321736,...,None,None,True,None,None,None,None,None,234.0,240.0
996,ff3ce693-ca96-4318-ba73-88f95acc5bc1,0.024092,0.279565,0.027082,0.087968,0.460908,0.121905,0.891634,0.180256,0.260631,...,True,True,None,None,None,None,None,None,108.0,114.0


In [75]:
# save the fused features in to csv
fused_features.to_csv("full_fp1_features.csv")

In [77]:
fused_features.columns

Index(['annotation_id', 'Unnamed: 0', 'BirdNET_0', 'BirdNET_1', 'BirdNET_2',
       'BirdNET_3', 'BirdNET_4', 'BirdNET_5', 'BirdNET_6', 'BirdNET_7',
       ...
       'rain or wind', 'unknown', 'nothing', 'parrot', 'drops', 'whistle',
       'sunbird', 'frog', 'start_time', 'end_time'],
      dtype='object', length=1038)

In [83]:
# dropping unwanted columns and add site information for the ml development
ml_features = fused_features.drop(columns = ['Unnamed: 0','bird', 'rain or wind', 'unknown', 'nothing','drops', 'whistle', 'sunbird', 'frog'])
ml_features['site']='fp1'

In [85]:
ml_features

,annotation_id,BirdNET_0,BirdNET_1,BirdNET_2,BirdNET_3,BirdNET_4,BirdNET_5,BirdNET_6,BirdNET_7,BirdNET_8,...,BirdNET_1019,BirdNET_1020,BirdNET_1021,BirdNET_1022,BirdNET_1023,path,parrot,start_time,end_time,site
0,003e93f7-cf43-40b9-8373-5d43c50871a8,0.077048,0.245560,0.255525,0.037137,0.617168,0.078559,0.528461,0.100497,0.186559,...,1.312904,0.487658,0.261881,0.000000,1.336080,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,True,282.0,288.0,fp1
1,00a47c33-c5fd-4a94-8f43-dd118660dde3,0.024353,0.668725,0.175876,0.258857,0.166176,0.081506,1.130004,0.199331,0.508516,...,1.007046,0.421046,0.377364,0.036960,1.248653,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,True,126.0,132.0,fp1
2,00c112fa-a272-484a-b7dc-ed242eb3af78,0.089544,0.452753,0.007244,0.233920,0.265389,0.122408,1.041941,0.143856,0.454708,...,0.891411,0.529044,0.370800,0.015614,1.336584,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,None,42.0,48.0,fp1
3,00d15e74-4ef8-445c-beff-d70c522640d7,0.139818,0.285597,0.051643,0.223714,0.291120,0.191062,1.089246,0.092381,0.409825,...,0.970368,0.350799,0.232303,0.000000,0.583674,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,None,24.0,30.0,fp1
4,0100972b-1c98-4089-b149-826f5d6d4808,0.084454,0.433399,0.059982,0.311007,0.316492,0.190456,0.915934,0.322653,0.485731,...,0.680800,0.480086,0.440197,0.003388,1.252957,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,None,36.0,42.0,fp1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993,fe4bf03c-db79-46bb-9f2e-d98523cddcde,0.003800,0.000000,0.184203,0.448457,0.917729,0.514493,0.364949,0.055889,0.288647,...,0.482356,0.481683,0.031169,0.108669,0.886504,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,None,96.0,102.0,fp1
994,fed25711-b2ae-432a-81e1-9a590d0a009e,0.103586,0.194146,0.178390,0.753330,0.427396,0.397441,0.545625,0.109222,0.447043,...,0.398920,0.648575,0.261855,0.000000,1.285823,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,None,204.0,210.0,fp1
995,ff15e62d-656e-4125-a476-dd6af2e20b6e,0.073153,0.489372,0.079489,0.416042,0.280941,0.060840,1.065122,0.234270,0.321736,...,1.066679,0.480850,0.471575,0.014748,1.322305,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,None,234.0,240.0,fp1
996,ff3ce693-ca96-4318-ba73-88f95acc5bc1,0.024092,0.279565,0.027082,0.087968,0.460908,0.121905,0.891634,0.180256,0.260631,...,0.409498,0.598947,0.337626,0.054441,1.057768,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,None,108.0,114.0,fp1


In [87]:
ml_features.to_csv("ml_fp1_features.csv")